## 네이버 이미지 크롤링
- selenium을 이용해 이미지 크롤링
- 폴더 생성/기본폴더, 파일명에 따른 다른 이름 저장

In [ ]:
# 필요한 모듈(라이브러리 연결)
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from urllib import request    # 이미지 다운로드에 사용
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import time
import os

#### 준비
1. 크롬 드라이브 실행
2. 검색어를 이용한 url 생성
3. 화면 스크롤 이동을 통한 추가 이미지 생성

In [ ]:
browser = Chrome('c:/pydata/chromedriver.exe')
time.sleep(1)

# 키워드 입력 및 url 생성
keyword=input("이미지 검색어 입력: ")

url="https://search.naver.com/search.naver?where=image&sm=tab_jum&query="
url += keyword  # url = url + keyword

# url을 이용한 페이지 이동
browser.get(url)

#스크로 이동 횟수 정의
count_down=int(input("스크롤 다운 횟수 입력(최대:6):"))

# 사용자가 요구한 스크롤 다운 횟수 확인(최대 6회)
if count_down > 6:
    count_down = 6


# windows.scrollTo(0, Height) : 스크롤을 아래쪽으로 이동
# Height 대신 document.body.scrollHeight를 입력하면 페이지 끝으로 이동

for i in range(count_down):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(1)

C:\Users\student\AppData\Local\Temp\ipykernel_376\1755421766.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = Chrome('c:/pydata/chromedriver.exe')


이미지 검색어 입력: 강아지
스크롤 다운 횟수 입력(최대:6):6


#### 화면에서 이미지 추출하기
1. 드라이브의 소스코드 받아오기
2. 이미지가 있는 태그 찾기
3. 태그에서 이미지 소스("src") 받아오기
4. src 주소를 이용해 이미지 다운로드 하기

In [ ]:
# 드라이브 소스코드 가져오기
html = browser.page_source
soup = bs(html, "html.parser")

#### Image lazy loading
- 현재 화면에 필요한 이미지만 불러오는 기능


In [ ]:
# 이미지 대표 태그 찾아 tag_img 변수에 저장(tag_img => 리스트형 데이터)
tag_img = soup.find_all("div", class_="thumb")

img=tag_img[4].find("img", class_="_image _listImage")

if img.get("data-lazy-src") == None:
    print(img["src"])
else:
    print(img.get("data-lazy-src"))

https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyMjAzMTVfMTg2%2FMDAxNjQ3Mjk2ODcyNjQ0.gQ2f9J7eHrGM0CA1LQz90Cl4KDCBEfcbVhLexWBU_TEg.Kiq8QX-05lcHlcMBwZqA2m_fshAI5-flw3fTG9KURlsg.JPEG.chajihyo%2F1647296873238.jpg&type=a340


In [ ]:
# 태그에서 이미지 소스("src") 받아오기
tag_src=[]
for tag in tag_img:
    img=tag.find("img", class_="_image _listImage")
    if img.get("data-lazy-src") == None:
        tag_src.append(tag.find("img")['src'])
    else:
        tag_src.append(img.get("data-lazy-src"))
        

#tag_src=[tag.find("img")['src'] for tag in tag_img]
#print(len(tag_src))
browser.close()

In [ ]:
# 이미지 src를 이용해 이미지 데이터 다운로드 후 저장하기
img_name = "c:/pydata/" + keyword + ".jpg"

from urllib import request

# 웹상의 이미지를 다운로드후 저장
# urllib.request.urlretrieve(대상 src_url, "저장경로와 파일명") 
request.urlretrieve(tag_src[4], img_name) 

('c:/pydata/강아지.jpg', <http.client.HTTPMessage at 0x29b3983df70>)

#### 검색어를 이용해 폴더 생성후 전체 이미지 저장
- os.makedirs("폴더 경로/생성 폴더명") => 폴더 생성 명령어
- 폴더가 없으면 생성후 저장
- 기존에 폴더가 존재하며 기존 폴더이름 뒤에 1, 2, .. 와 같이 번호를 붙여 생성 


In [1]:
# 폴더 생성

os.makedirs("c:/pydata/imgs")
os.listdir('c:/pydata/')

NameError: ignored

In [ ]:
os.path.exists("c:/pydata/")   # 폴더 존재여부 확인: 있으면 True 반환, 없으면 False 반환

True

In [ ]:
# 이미지를 저장할 대표 폴더 생성(images)
fdir="c:/pydata/images"

if os.path.exists(fdir):  # 폴더가 있다면 뒤쪽에 "/"만 연결
    fdir += "/"
else:
    os.makedirs(fdir)     # 폴더가 없다면 폴더 생성후 뒤쪽에 "/"만 연결
    fdir += "/"

# images 폴더에 검색 키워드를 이용해 폴더 생성후 저장
# 키워드와 동일한 폴더가 있는 경우 뒤에 1씩 번호를 증가시기면서 폴더명 확인(없으면 생성)

if not os.path.exists(fdir + keyword):
    os.makedirs(fdir + keyword)     # 폴더가 없다면 폴더 생성후 뒤쪽에 "/"만 연결
    fdir = fdir + keyword +  "/"
else:
    # 폴더가 있다면 새로운 폴더 생성(번호 증가)
    num = 0
    while True:
        num += 1 #번호 1씩 증가
        
        #증가된 번호와 기존 폴더명을 연경해서 존재여부 확인
        if not os.path.exists(fdir + keyword + str(num)):
            # 없으면 폴더 생성후 while 종료
            os.makedirs(fdir + keyword + str(num)) 
            fdir = fdir + keyword + str(num) + "/"
            break        

이미지 저장
- 저장 이미지 이름 뒤에 번호 붙이기

In [ ]:
num = 1
for tag in tqdm(tag_src):
    img_name = fdir + keyword + str(num) + ".jpg"
    request.urlretrieve(tag, img_name)
    #time.sleep(1)
    num += 1

100%|████████████████████████████████████████████████████████████████████████████████| 545/545 [00:20<00:00, 26.44it/s]


In [ ]:
tag_src[4]

'data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7'